In [68]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from helita.sim import rh15d 
from helita.vis import rh15d_vis

from scipy.integrate import quadrature
from scipy.integrate import cumtrapz

from pathlib import Path

rhRepoPath = Path("/Users/elka127/Documents/code/rh")

# Atoms for CaII

```
# Nmetal
   10

# Metals
#  model file              ACTIVE/PASSIVE  INITIAL_SOLUTION   population file


# ../../Atoms/CaII.atom            PASSIVE    ZERO_RADIATION
  ../../Atoms/H_6.atom             PASSIVE    ZERO_RADIATION
  ../../Atoms/MgII-IRIS.atom       PASSIVE    ZERO_RADIATION
  ../../Atoms/CaII_PRD.atom        ACTIVE     ZERO_RADIATION
  ../../Atoms/Si.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/Al.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/Fe.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/He.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/N.atom               PASSIVE    LTE_POPULATIONS
  ../../Atoms/Na.atom              PASSIVE    LTE_POPULATIONS
  ../../Atoms/S.atom               PASSIVE    LTE_POPULATIONS
```

In [2]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 392.8) & (wave < 394.0)]

wave.sel(wavelength=500, method="nearest")
index500 = np.argmin(np.abs(wave.data - 500))

with open(rhRepoPath / "rh15d/run/ray.input", "w") as f:
    f.write("1.00\n")
    output = str(len(indices) + 1)
    for ind in indices:
        output += f" {ind}"
    output += f" {index500}\n"
    f.write(output)

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.
/Users/elka127/Documents/code/helita/helita/sim/rh15d.py:40: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  setattr(self, g, xr.open_dataset(infile, group=g, autoclose=True))
/Users/elka127/Documents/code/helita/helita/sim/rh15d.py:53: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  self.ray 

# Intencity plot for Ca II

In [69]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.
/Users/elka127/Documents/code/helita/helita/sim/rh15d.py:40: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  setattr(self, g, xr.open_dataset(infile, group=g, autoclose=True))
/Users/elka127/Documents/code/helita/helita/sim/rh15d.py:53: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  self.ray 

In [72]:
data.ray.intensity.plot()
plt.axis([392.8, 394.0, 0, 2e-9])

ValueError: The input coordinate is not sorted in increasing order along axis 0. This can lead to unexpected results. Consider calling the `sortby` method on the input DataArray. To plot data with categorical axes, consider using the `heatmap` function from the `seaborn` statistical plotting library.

# Calculating optical depths

In [73]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
print(data.ray.wavelength_selected)
print("-")
print(data.ray.wavelength_selected.data)

# first column
height = data.atmos.height_scale[0, 0].dropna('height')  
# index of 500 nm
index500 = np.argmin(np.abs(data.ray.wavelength_selected.data - 500))  

tau500 = cumtrapz(data.ray.chi[0, 0, :, index500].dropna('height'), x=-height)
# ensure tau500 and height have same size
tau500 = np.concatenate([[1e-20], tau500])  

 

/Users/elka127/Documents/code/helita/helita/sim/rh15d.py:40: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  setattr(self, g, xr.open_dataset(infile, group=g, autoclose=True))
/Users/elka127/Documents/code/helita/helita/sim/rh15d.py:53: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  self.ray = xr.open_dataset(infile, autoclose=True)
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/

In [74]:
fig, ax = plt.subplots()
ax.plot(height / 1e6, tau500)  # height in Mm
ax.set_xlabel('H (Mm)')
ax.set_ylabel(r'$\tau$$_{500}$')
ax.set_yscale('log')
ax.axhline(y=1,linestyle='dashed',color='black')
ax.axvline(x=0.5,linestyle='dashed',color='black')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Plot the departure coefficients for the levels of Ca II



In [8]:
# plt.figure()
# for i in range(5):
#     plt.plot(data.atmos.height_scale[0,0]/1e6, 
#              data.atom_CA.populations[i, 0, 0]/data.atom_CA.populations_LTE[i, 0, 0],
#              label='Level %i' % (i + 1))
# plt.legend(loc="upper left")
# plt.figure()
# for i in range(5):
#     plt.plot(data.atmos.height_scale[0,0]/1e6, 
#              data.atom_CA.populations[i, 0, 0]/data.atom_CA.populations_LTE[i, 0, 0],
#              label='Level %i' % (i + 1))
fig, ax = plt.subplots()
ax.plot(tau500,data.atom_CA.populations[1, 0, 0]/data.atom_CA.populations_LTE[1, 0, 0], label='Level %1' % (1)) 
plt.legend(loc="upper left")
plt.xlabel("tau500")
plt.ylim(0.05, 2)
plt.ylabel("Departure coefficients")
plt.ylim(0.05, 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: incomplete format

# от тау 500

In [61]:
# xVals = data.atmos.height_scale[0,0]/1e6
# with open("/tmp/x.txt", "w") as f:
#     for x in xVals.data:
#         f.write(f"{x}\n")

print(data.atom_CA.populations[0,22,0])
# yVals = data.atom_CA.populations[0,0,0]
# with open("/tmp/y.txt", "w") as f:
#     for y in yVals.data:
#         f.write(f"{y}\n")


#print(data.atom_CA.populations[0,0,0][247])
# print("- - - - -")
# print(data.atom_CA.populations[1,0,0])
# print("- - - - -")
# print(data.atom_CA.populations[2,0,0])

<xarray.DataArray 'populations' (height: 249)>
array([7.636049e-01, 7.605768e-01, 7.429523e-01, ..., 3.095008e+18,
       2.967401e+18, 2.879468e+18], dtype=float32)
Coordinates:
    x        float64 32.0
    y        float64 10.0
Dimensions without coordinates: height


In [87]:
import matplotlib.animation as animation

fig, ax = plt.subplots()

line, = ax.plot(
    data.atmos.height_scale[0,0]/1e6,
    data.atom_CA.populations[0,0,0]
)

def animate(i):
    #line.set_xdata(x)
    line.set_ydata(data.atom_CA.populations[0, i, 0][0])
    return line,

ani = animation.FuncAnimation(
    fig,
    animate,
    np.arange(52),
    #blit=True,
    repeat=False
)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
plt.figure()
for i in range(5):
    plt.plot(
        data.atmos.height_scale[0,0]/1e6, 
        data.atom_CA.populations[i, 51, 0]/data.atom_CA.populations_LTE[i, 51, 0],
        label='Level %i' % (i + 1)
    )
plt.legend(loc="upper left")
plt.xlabel("Height (Mm)")
plt.ylabel("Departure coefficients")
plt.xlim(0,11)
plt.ylim(-20,10000)
#/data.atom_CA.populations_LTE[i, 100, 0]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-20.0, 10000.0)

# Source function

In [82]:
rh15d_vis.SourceFunction(data);

/usr/local/lib/python3.8/site-packages/astropy/units/quantity.py:477: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='wavelength', max=43), Checkbox(value=True, description='…